In [ ]:
def predict(model, img_paths, stride=32, batch_size=128, img_size=256): #img_paths => GT
    results = []
    for img_path in img_paths:
        img = cv2.imread(img_path)
        img = img.astype(np.float32)/255
        crop = []
        position = []
        batch_count = 0
        img = img.transpose(2,0,1)
        print(img.shape)
        result_img = np.zeros_like(img)
        voting_mask = np.zeros_like(img)
        model.eval()
        with torch.no_grad():
            for top in tqdm.tqdm(range(0, img.shape[1], stride)): 
                for left in range(0, img.shape[2], stride):
                    piece = np.zeros([3, img_size, img_size], np.float32)
                    temp = img[:, top:top+img_size, left:left+img_size]
                    piece[:,:temp.shape[1], :temp.shape[2]] = temp
                    crop.append(piece)
                    position.append([top, left])
                    batch_count += 1
                    if batch_count == 40:
                        crop = torch.tensor(crop).to(device)
                        pred = model(crop)
                        crop = []
                        batch_count = 0
                        for num, (t, l) in enumerate(position):
                            piece = pred[num]
                            c, h, w = result_img[:, t:t+img_size, l:l+img_size].shape
                            _piece = np.array((piece[:,:h, :w]*255).cpu().detach(),dtype=np.uint8)
                            result_img[:, t:min(t+img_size,img.shape[1]), l:min(l+img_size,img.shape[2])] += _piece[:, :h, :w]
                            voting_mask[:, t:t+img_size, l:l+img_size] += 1
                        position = []

            result_img = result_img/voting_mask
            result_img = result_img.astype(np.uint8)
            print(result_img.shape)
            print(np.max(result_img))
            results.append(result_img)
    return results
